In [1]:
from selenium import webdriver
import pandas as pd
import numpy as np
from selenium.webdriver.common.by import By

In [2]:
# The jobs page automatically defaults to your location. Therefore, it turns up 0 results.
# The below code will help with making the jobs page not default to a specific locaion.

driver = webdriver.Chrome()
driver.implicitly_wait(10)

url = 'https://careers.walmart.com/'
driver.get(url)

job_search = driver.find_element(By.XPATH, '//*[@id="search"]')
job_search.send_keys('technology')

search_button = driver.find_element(By.XPATH, '//*[@id="location"]')


search_button.click()

In [3]:
#job_num = []
TITLE = []
LINK = []

y = 1
i = 1

while y != 0:  
#while i < 3:  
    
    try:
        
        url = 'https://careers.walmart.com/results?q=&page='+str(i)+'&sort=rank&jobCategory=00000161-7bad-da32-a37b-fbef5e390000,00000161-7bf4-da32-a37b-fbf7c59e0000,00000161-7bff-da32-a37b-fbffc8c10000,00000161-8bd0-d3dd-a1fd-bbd0febc0000,00000161-8be6-da32-a37b-cbe70c150000&jobSubCategory=0000015a-a577-de75-a9ff-bdff284e0000&expand=department,0000015e-b97d-d143-af5e-bd7da8ca0000,00000161-8be6-da32-a37b-cbe70c150000,brand,type,rate&type=jobs'
        
        driver.get(url) 
    
        for j in range(1,26): 

            job = driver.find_element(By.XPATH, '//*[@id="search-results"]/li['+str(j)+']/div[1]/h4/a')
            #print(job)

            try:
                TITLE.append(job.text)
            except:
                TITLE.append('')

            try:
                LINK.append(job.get_attribute('href'))
            except:
                LINK.append('')     
        
    except:
        y -= 1
    
    #print('page', i)
    
    i += 1
    
driver.close()    
print(len(TITLE), len(LINK)) 


408 408


In [4]:
print(TITLE[1:10])

['Senior Manager II, Technical Program Management', '(USA) Staff Technical Program Manager - Tech', 'Staff Technical Program Manager- Information Security Business', '(USA) Senior Director II, Technical Program Management', '(USA) Senior Fuel Operations Manager - WM NBD', 'Principal Technology Operations', 'Staff Technology Operations', '(USA) Senior Manager II, Technology Operations - Customer Advocacy Program', 'Senior Manager II, Omnichannel & eCommerce Strategy (Principal Technology Operations)']


In [5]:
print(LINK[1:5])

['https://careers.walmart.com/us/jobs/WD1412797-senior-manager-ii-technical-program-management', 'https://careers.walmart.com/us/jobs/WD1379878-usa-staff-technical-program-manager-tech', 'https://careers.walmart.com/us/jobs/WD1391299-staff-technical-program-manager-information-security-business', 'https://careers.walmart.com/us/jobs/WD1416415-usa-senior-director-ii-technical-program-management']


In [6]:
df = pd.DataFrame(zip(TITLE, LINK))
df['QUALIFICATIONS'] = np.nan


In [7]:
df.columns = ['TITLE', 'LINK', 'QUALIFICATIONS']
df.head()

,TITLE,LINK,QUALIFICATIONS
0,Staff Technical Program Manager,https://careers.walmart.com/us/jobs/WD1397133-...,NaN
1,"Senior Manager II, Technical Program Management",https://careers.walmart.com/us/jobs/WD1412797-...,NaN
2,(USA) Staff Technical Program Manager - Tech,https://careers.walmart.com/us/jobs/WD1379878-...,NaN
3,Staff Technical Program Manager- Information S...,https://careers.walmart.com/us/jobs/WD1391299-...,NaN
4,"(USA) Senior Director II, Technical Program Ma...",https://careers.walmart.com/us/jobs/WD1416415-...,NaN


In [8]:
df = df.drop_duplicates(subset=['TITLE'])
len(df)

222

In [9]:
df.to_csv('walmart_jobs_title_n_link_updated012723.csv')

In [10]:
driver = webdriver.Chrome()

for i in range(len(df['LINK'])):

    try:
        
        url = (df['LINK'][i])

        driver.get(url)

        desc = driver.find_element(By.XPATH, '/html/body/main/section[3]/div/div[2]')
        

        df['QUALIFICATIONS'][i] = desc.text   
         
    except:
        df['QUALIFICATIONS'][i] = np.nan

driver.close()
        


C:\Users\meske\AppData\Local\Temp/ipykernel_9032/2761919586.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['QUALIFICATIONS'][i] = desc.text


In [11]:
df.isna().sum()

TITLE              0
LINK               0
QUALIFICATIONS    70
dtype: int64

In [13]:
df = df.drop_duplicates(subset=['TITLE', 'LINK', 'QUALIFICATIONS'])
df['QUALIFICATIONS'] = df['QUALIFICATIONS'].str.lower()
df = df.dropna()
df['COMPANY'] = 'Walmart'
df = df.reset_index(drop = True)
len(df)

152

In [15]:
# removing unnecessary text

desc = df['QUALIFICATIONS'][1]

mid = desc.index('minimum qualifications')+22

desc = desc[mid:]

print(desc)



...
outlined below are the required minimum qualifications for this position. if none are listed, there are no minimum qualifications.
• option 1: bachelor’s degree in computer science, information technology, engineering, or related area and 6 years’ experience in engineering, engineering program management, technical program management, product management, or related area.
• option 2: 8 years’ experience in engineering, engineering program management, technical program management, product management, or related area.
preferred qualifications...
outlined below are the optional preferred qualifications for this position. if none are listed, there are no preferred qualifications.
certification in project management., master’s degree in business administration, with specialization in strategy, supply chain, finance, information systems, or related area and 4 years’ experience in product design., supervisory
primary location...
805 se moberly ln, bentonville, ar 72712, united states of am

In [16]:
for i in range(len(df['QUALIFICATIONS'])):

    try:

        desc = df['QUALIFICATIONS'][i]

        mid = desc.index('minimum qualifications')+22

        desc = desc[mid:]

        df['QUALIFICATIONS'][i] = desc

    except:
        pass


In [17]:
df['QUALIFICATIONS'][1]

'...\noutlined below are the required minimum qualifications for this position. if none are listed, there are no minimum qualifications.\n• option 1: bachelor’s degree in computer science, information technology, engineering, or related area and 6 years’ experience in engineering, engineering program management, technical program management, product management, or related area.\n• option 2: 8 years’ experience in engineering, engineering program management, technical program management, product management, or related area.\npreferred qualifications...\noutlined below are the optional preferred qualifications for this position. if none are listed, there are no preferred qualifications.\ncertification in project management., master’s degree in business administration, with specialization in strategy, supply chain, finance, information systems, or related area and 4 years’ experience in product design., supervisory\nprimary location...\n805 se moberly ln, bentonville, ar 72712, united sta

In [18]:
# removing unnecessary text

remove1 = 'outlined below are the required minimum qualifications for this position. if none are listed, there are no minimum qualifications'
remove2 = 'outlined below are the optional preferred qualifications for this position. if none are listed, there are no preferred qualifications'

In [19]:
for i in range(len(df['QUALIFICATIONS'])):

    try:

        df['QUALIFICATIONS'][i] = df['QUALIFICATIONS'][i].replace(remove1, '')
        
    except:
        pass

In [20]:
df['QUALIFICATIONS'][1]

'...\n.\n• option 1: bachelor’s degree in computer science, information technology, engineering, or related area and 6 years’ experience in engineering, engineering program management, technical program management, product management, or related area.\n• option 2: 8 years’ experience in engineering, engineering program management, technical program management, product management, or related area.\npreferred qualifications...\noutlined below are the optional preferred qualifications for this position. if none are listed, there are no preferred qualifications.\ncertification in project management., master’s degree in business administration, with specialization in strategy, supply chain, finance, information systems, or related area and 4 years’ experience in product design., supervisory\nprimary location...\n805 se moberly ln, bentonville, ar 72712, united states of america'

In [21]:
for i in range(len(df['QUALIFICATIONS'])):

    try:

        df['QUALIFICATIONS'][i] = df['QUALIFICATIONS'][i].replace(remove2, '')
        
    except:
        pass

In [22]:
df['QUALIFICATIONS'][1]

'...\n.\n• option 1: bachelor’s degree in computer science, information technology, engineering, or related area and 6 years’ experience in engineering, engineering program management, technical program management, product management, or related area.\n• option 2: 8 years’ experience in engineering, engineering program management, technical program management, product management, or related area.\npreferred qualifications...\n.\ncertification in project management., master’s degree in business administration, with specialization in strategy, supply chain, finance, information systems, or related area and 4 years’ experience in product design., supervisory\nprimary location...\n805 se moberly ln, bentonville, ar 72712, united states of america'

In [23]:
df.head()

,TITLE,LINK,QUALIFICATIONS,COMPANY
0,Staff Technical Program Manager,https://careers.walmart.com/us/jobs/WD1397133-...,...\n.\n• option 1: bachelor’s degree in compu...,Walmart
1,"Senior Manager II, Technical Program Management",https://careers.walmart.com/us/jobs/WD1412797-...,...\n.\n• option 1: bachelor’s degree in compu...,Walmart
2,(USA) Staff Technical Program Manager - Tech,https://careers.walmart.com/us/jobs/WD1379878-...,...\n.\nminimum qualifications: bachelor of sc...,Walmart
3,Staff Technical Program Manager- Information S...,https://careers.walmart.com/us/jobs/WD1391299-...,...\n.\n• option 1: bachelor’s degree in compu...,Walmart
4,"(USA) Senior Director II, Technical Program Ma...",https://careers.walmart.com/us/jobs/WD1416415-...,...\n.\n• option 1: bachelor’s degree in compu...,Walmart


In [24]:
df['DESCRIPTION'] = df['QUALIFICATIONS']
df = df[['COMPANY', 'TITLE', 'QUALIFICATIONS', 'LINK', 'DESCRIPTION']]
df.head()

,COMPANY,TITLE,QUALIFICATIONS,LINK,DESCRIPTION
0,Walmart,Staff Technical Program Manager,...\n.\n• option 1: bachelor’s degree in compu...,https://careers.walmart.com/us/jobs/WD1397133-...,...\n.\n• option 1: bachelor’s degree in compu...
1,Walmart,"Senior Manager II, Technical Program Management",...\n.\n• option 1: bachelor’s degree in compu...,https://careers.walmart.com/us/jobs/WD1412797-...,...\n.\n• option 1: bachelor’s degree in compu...
2,Walmart,(USA) Staff Technical Program Manager - Tech,...\n.\nminimum qualifications: bachelor of sc...,https://careers.walmart.com/us/jobs/WD1379878-...,...\n.\nminimum qualifications: bachelor of sc...
3,Walmart,Staff Technical Program Manager- Information S...,...\n.\n• option 1: bachelor’s degree in compu...,https://careers.walmart.com/us/jobs/WD1391299-...,...\n.\n• option 1: bachelor’s degree in compu...
4,Walmart,"(USA) Senior Director II, Technical Program Ma...",...\n.\n• option 1: bachelor’s degree in compu...,https://careers.walmart.com/us/jobs/WD1416415-...,...\n.\n• option 1: bachelor’s degree in compu...


In [26]:
df.to_csv('walmart_technology_jobs.csv')

In [27]:
df.columns

Index(['COMPANY', 'TITLE', 'QUALIFICATIONS', 'LINK', 'DESCRIPTION'], dtype='object')